# base line script

1. anacoda를 설치
2. jupyter notebook 사용법을 익혀야함
3. pandas로 데이터 전처리에 대해 익혀야함
4. sci-kit learn lib을 통해 머신러닝 알고리즘 적용을 익혀야함
5. model tunning 과 validation을 이해하고 다시 3으로 돌아가서 반복적으로 실험을 할 수 있어야함

In [113]:
import pandas as pd
import numpy as np


## load data

In [114]:
file_num = 1
merged_df = None

In [115]:
for each_file in range(file_num):
        df = pd.read_csv('%d_2016-11-14.csv' % each_file, index_col='date', parse_dates=['date'])
        if merged_df is None:
            merged_df = pd.DataFrame(columns=df.columns)
        df.sort_index(inplace=True)
        merged_df = pd.concat([merged_df, df])

In [116]:
merged_df.head(3)

,Unnamed: 0,open,high,low,close,sign,change,diff,volume,diff_vol,...,o_sign,o_change,o_diff,h_sign,h_change,h_diff,l_sign,l_change,l_diff,marketcap
2013-03-20,899.0,935.0,938.0,905.0,922.0,5.0,16.0,-1.71,1166500.0,-12.47,...,5.0,-3.0,-0.32,3.0,0.0,0.00,5.0,-33.0,-3.52,39554.0
2013-03-21,898.0,906.0,937.0,906.0,918.0,5.0,4.0,-0.43,572162.0,-50.95,...,5.0,-16.0,-1.74,2.0,15.0,1.63,5.0,-16.0,-1.74,39382.0
2013-03-22,897.0,918.0,934.0,903.0,906.0,5.0,12.0,-1.31,475689.0,-16.86,...,3.0,0.0,0.00,2.0,16.0,1.74,5.0,-15.0,-1.63,38867.0


* 같은 디렉토리내의 csv 파일을 pandas dataframe으로 가져온 것임
* 보면 Unnamed와 같은 쓸모 없는 칼럼이 있음 드랍해줍시다

In [117]:
merged_df.drop(['Unnamed: 0', 'shcode'], axis=1, inplace=True)

In [118]:
merged_df.head(3)

,open,high,low,close,sign,change,diff,volume,diff_vol,chdegree,...,o_sign,o_change,o_diff,h_sign,h_change,h_diff,l_sign,l_change,l_diff,marketcap
2013-03-20,935.0,938.0,905.0,922.0,5.0,16.0,-1.71,1166500.0,-12.47,51.99,...,5.0,-3.0,-0.32,3.0,0.0,0.00,5.0,-33.0,-3.52,39554.0
2013-03-21,906.0,937.0,906.0,918.0,5.0,4.0,-0.43,572162.0,-50.95,63.02,...,5.0,-16.0,-1.74,2.0,15.0,1.63,5.0,-16.0,-1.74,39382.0
2013-03-22,918.0,934.0,903.0,906.0,5.0,12.0,-1.31,475689.0,-16.86,59.52,...,3.0,0.0,0.00,2.0,16.0,1.74,5.0,-15.0,-1.63,38867.0


## data preprocessing
1. 이제 데이터 전처리를 해야하는데 중간에 미싱데이터는 있는지 정규화를 할지 등등을 판단하고 처리해야합니다.
2. 위에서 한 정렬과 컬럼제거 작업도 전처리에 포홤됩니다.

In [119]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900 entries, 2013-03-20 to 2016-11-14
Data columns (total 26 columns):
open          900 non-null float64
high          900 non-null float64
low           900 non-null float64
close         900 non-null float64
sign          900 non-null float64
change        900 non-null float64
diff          900 non-null float64
volume        900 non-null float64
diff_vol      900 non-null float64
chdegree      900 non-null float64
sojinrate     900 non-null float64
changerate    900 non-null float64
fpvolume      900 non-null float64
covolume      900 non-null float64
value         900 non-null float64
ppvolume      900 non-null float64
o_sign        900 non-null float64
o_change      900 non-null float64
o_diff        900 non-null float64
h_sign        900 non-null float64
h_change      900 non-null float64
h_diff        900 non-null float64
l_sign        900 non-null float64
l_change      900 non-null float64
l_diff        900 non-null float64
m

* 전체 데이터가 3만개 정도 이고 널데이터는 없는 것을 알 수 있음
* 무엇을 예측 해볼 것인가? 간단하게 sign(그날 오르고 내렸나를 값으로 가지고 있음 1:상한, 2:상승, 3:보합, 4:하한, 5:하락)을 예측해봅시다.
* 현재 sign은 그날 당시를 표현하니 그 다음 날의 sign을 가지고 있다면 맞추고자 하는 레이블을 가지고 있다고 할수 있죠  

In [120]:
merged_df['y']=merged_df['sign'].shift(-1)

In [121]:
merged_df[['sign','y']].tail(3)

,sign,y
2016-11-10,2.0,2.0
2016-11-11,2.0,3.0
2016-11-14,3.0,NaN


In [122]:
merged_df.dropna(inplace=True)

In [123]:
merged_df[['sign','y']].tail(3)

,sign,y
2016-11-09,5.0,2.0
2016-11-10,2.0,2.0
2016-11-11,2.0,3.0


## feature egineering
1. 여기가 결국은 모델의 성능 즉. 퍼포먼스와 직결되는 부분인데 아이디어가 필요합니다.
2. 예를 들어 위 데이터는 시계열기반의 데이터인데 시계열 특성을 나타내는 컬럼이 부족합니다. 어떻게 시계열을 반영할지 생각해야함

## modeling
1. 위에서 처리가 다 끝났으면 이제 학습을 시작해봅시다.
2. 데이터 특성에 기반해서 어떤 머신러닝 모델을 쓸지 판단할 수 있지만 결국은 실험을 해봐야함

In [124]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

In [130]:
features = merged_df.columns.drop(['y'])
print(features)
label = 'y'
train_X = merged_df[features]
train_y = merged_df[label]

Index(['open', 'high', 'low', 'close', 'sign', 'change', 'diff', 'volume',
       'diff_vol', 'chdegree', 'sojinrate', 'changerate', 'fpvolume',
       'covolume', 'value', 'ppvolume', 'o_sign', 'o_change', 'o_diff',
       'h_sign', 'h_change', 'h_diff', 'l_sign', 'l_change', 'l_diff',
       'marketcap'],
      dtype='object')


In [1]:
svc_param = {'C':np.logspace(-2,0,20)}

NameError: name 'np' is not defined

In [127]:
gs_svc = GridSearchCV(LinearSVC(loss='l2'),svc_param,cv=5,n_jobs=4,verbose=1)

In [128]:
gs_svc.fit(np.array(train_X),np.array(train_y))

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/svm/classes.py:199: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/svm/classes.py:199: DeprecationWarning: loss='l2' has been deprecated in favor of loss='squared_hinge' as of 0.16. Backward compatibility for the loss='l2' will be removed in 1.0
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/svm/classes.py:199: DeprecationWarn

GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='l2', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01274,  0.01624,  0.02069,  0.02637,  0.0336 ,
        0.04281,  0.05456,  0.06952,  0.08859,  0.11288,  0.14384,
        0.1833 ,  0.23357,  0.29764,  0.37927,  0.48329,  0.61585,
        0.78476,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [129]:
print (gs_svc.best_params_, gs_svc.best_score_)

{'C': 0.61584821106602605} 0.4682981090100111


* 지금 하는 작업은 학습을 하면서 최적화된 파라미터를 구하는 과정입니다. 다 설명하기 넘나 귀찮 ㅠ 찾아보세요 무엇을 하는지
* 위에보이는 는 cross-validation-value입니다. 이게 높으면 일반화된 좋은 모델이라는 뜻이죠 트레이닝 스코어는 어떻게 구할까요?
* 모두 sklearn에 함수가 있습니다. 구해봐
* 결국 cv값과 트레이닝값을 비교해서 어떻게 feature engineering을 할지 생각하고 반복해야합니다. feature engineering으로 돌아가세요
* 참고로 학습된 모델을 사용하는 방법도 있는데 찾아보세요 ㅋㅋ